<h3>Requirements and import</h3>

In [ ]:
!git clone https://www.github.com/GEM-benchmark/NL-Augmenter
!cd /home/uccollab/Desktop/NL-Augmenter
!python setup.py sdist
!pip install -e .

In [ ]:
!pip install -r requirements.txt --quiet
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

In [ ]:
!pip install torch==1.7.1
!pip install checklist
!pip install sacrebleu
!pip install torchtext==0.8.0
!pip install benepar

In [ ]:
!pip install sacremoses

In [ ]:
'''This is necessary on some devices to avoid conflicts in NL-augmenter'''
cd /path/to/NL-Augmenter/
cd /path/to/NL-Augmenter/nlaugmenter/transformations/factive_verb_transformation

In [3]:
import nlaugmenter

In [4]:
from nlaugmenter.transformations.factive_verb_transformation import FactiveVerbTransformation
from nlaugmenter.transformations.formality_change import Formal2Casual
from nlaugmenter.transformations.replace_with_hyponyms_hypernyms import ReplaceHypernyms
from nlaugmenter.transformations.style_paraphraser import StyleTransferParaphraser
from nlaugmenter.transformations.synonym_substitution import SynonymSubstitution
from nlaugmenter.transformations.back_translation import BackTranslation
from nlaugmenter.transformations.filler_word_augmentation import FillerWordAugmentation
from nlaugmenter.transformations.protaugment_diverse_paraphrase import ProtaugmentDiverseParaphrase
from nlaugmenter.transformations.slangificator import Slangificator
from nlaugmenter.transformations.factive_verb_transformation import *

2022-05-20 23:13:08.710371: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [5]:
import itertools, re, time
import pprint as pp
import pandas as pd
import ast
import copy
import csv

<h3>Wrapping augmentation techniques that require utterance splitting</h3>

In [6]:
#replaces common nouns with other related words that are either hyponyms or hypernyms
class HyperNymAugmentation():
    
    def generate(self, utterance):
        tr = ReplaceHypernyms()
        result = []
        if ". " in utterance:
            for sentence in utterance.split(". "):
                aug_pool = tr.generate(sentence)
                for aug_sentence in aug_pool:
                    result.append(aug_sentence)
        else:
            aug_pool = tr.generate(utterance)
            for aug_sentence in aug_pool:
                result.append(aug_sentence)
        
        result = list(itertools.chain.from_iterable(result))
        result = "".join(result).replace('\\','')
        result = re.sub('\n+', ' ', result)
        return result

In [7]:
#replaces common nouns with other related words that are either hyponyms or hypernyms
class SentenceAdd():
    def generate(self, utterance):
        tr = SentenceAdditions()
        result = []
        if ". " in utterance:
            for sentence in utterance.split(". "):
                    result.append(tr.generate(sentence))
        else:
            result.append(tr.generate(utterance))
        result = list(itertools.chain.from_iterable(result))
        result = "".join(result).replace('\\','')
        result = re.sub('\n+', ' ', result)
        return result

<h3>Opening AnnoMI and pre-processing</h3>

In [ ]:
'''PRE-PROCESSING
- Loading AnnoMI
- Filter on therapist utterances
- Mapping therapy quality to int
- cutting out short utterances
''' 
df= pd.read_csv("/path/to/dataset.csv")
df = df[df['interlocutor'] == 'therapist']
df['mi_quality'] = df['mi_quality'].map({'high':1, "low":0})
df = df[df.columns[df.columns.isin(['mi_quality', 'utterance_text', 'topic'])]]
df = df[df['utterance_text'].apply(lambda x: len(x.split()) >5 )]

In [ ]:
'''Sampling 400 HQ utterances and 100 LW ones for testset''' 

#### import random
import pprint

hq = df[df['mi_quality']==1].to_dict('index')
lq = df[df['mi_quality']==0].to_dict('index')
df = df.to_dict('index')

hq = [v for k,v in hq.items()]
lq = [v for k,v in lq.items()]
df = [v for k,v in df.items()]

hq = random.sample(hq, 400)
lq = random.sample(lq,100)

indexes = []
hq_utter = [el['utterance_text'] for el in hq]
lq_utter = [el['utterance_text'] for el in lq]

i = 0
for el in df:
    utterance = el['utterance_text']
    if utterance in hq_utter or utterance in lq_utter:
        indexes += [i]
    i +=1

df = [v for i,v in enumerate(df) if i not in frozenset(indexes)] 

df = pd.DataFrame(df)
hq = pd.DataFrame(hq)
lq = pd.DataFrame(lq)
test_set = pd.concat([hq,lq]).reset_index(drop=True)


In [ ]:
'''Saving the files''' 

test_set.to_csv('/path/to/test-set.csv')
df.to_csv('/path/to/anno_mi_therapist_only.csv')

<h3>Instantiating the augmentation pipeline</h3>

In [8]:
cd /home/uccollab/Desktop/NL-Augmenter/nlaugmenter/transformations/factive_verb_transformation

/home/uccollab/Desktop/NL-Augmenter/nlaugmenter/transformations/factive_verb_transformation


In [9]:
'''Check below for information on each element. Computationally intensive ones are omitted.'''

start_time = time.time()
print("Instantiating augmenters, this may take a while...")
augmenters = [(FactiveVerbTransformation(),"FactiveVerb"), 
              (Formal2Casual(),"Formal2Casual"), 
              #(LostInTranslation(),"LostInTranslation"), 
              (HyperNymAugmentation(),"Hypernym substitution"),
              (StyleTransferParaphraser(style="Basic"),"Basic style"),
              (StyleTransferParaphraser(style="Tweets"),"Tweet style"),
              (SynonymSubstitution(),"Synonym substitution"),
              (BackTranslation(),"Backtranslation"),
              (FillerWordAugmentation(),"Filler Word"),
              (ProtaugmentDiverseParaphrase(),"ProtAugment"),
              #(SentenceAdd(),"Sentence Add"),
              (Slangificator(),"Slangificator"), 
              #(ParaphraseSowReap(max_outputs=4),"Sow Reap")
             ]
print(f"Augmenters instantiated ({round(time.time()-start_time,2)} seconds)")

Instantiating augmenters, this may take a while...


Some weights of the model checkpoint at filco306/gpt2-base-style-paraphraser were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.weight', 'transformer.extra_embedding_project.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at filco306/gpt2-tweet-paraphraser were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.weight', 'transformer.extra_embedding_project.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on anoth

Augmenters instantiated (54.75 seconds)


<h3>Target-based augmentation</h3>
The following is a naive augmentation, with the goal of balancing HQ and LQ utterances. It iterates the dataset and augments the minority class (LQ in the case of AnnoMI) until balance is reached.

This procedure results in the <b>Anno-AugMI</b> dataset

In [ ]:
df= pd.read_csv("/path/to/anno_mi_therapist_only.csv")

df_lo = df[df["mi_quality"]==0]
df_hi = df[df["mi_quality"]==1]

print(f'LQ: {df_lo.shape[0]}')
print(f'HQ: {df_hi.shape[0]}')

df_lo = df_lo.to_dict('index')
df_lo = [v for k,v in df_lo.items()]
print(f'Total utter.: {len(df)}')

In [ ]:
'''For analysis and debugging reasons, this procedure introduces an additional column, indicating
for each augmentation, the pipeline element that produced it.'''

augmentation_result = []
aug_counter = 0
row_counter = 0
aug_data = {}
print("Beginning augmentation")
for row in df_lo: 
    used_augmenters = []
    utterance = row['utterance_text']
    row_counter += 1
    final_result = []
    print(f"Augmenting row {row_counter}/{len(df_lo)}")
    print(f"\n{utterance}\n")
    row_start_time = time.time()
    for augmenter, name in augmenters:
        aug_counter += 1
        aug_start_time = time.time()
        try:
            aug_result = augmenter.generate(utterance)
            if type(aug_result) == list:
                for aug_utterance in aug_result:
                    if aug_utterance != utterance:
                        aug_utterance = aug_utterance.replace('\\','')
                        aug_utterance = re.sub('\+', ' ', aug_utterance)
                        final_result.append(aug_utterance)
                        used_augmenters.append(name)
            elif aug_result != utterance:
                aug_result = aug_result.replace('\\','')
                final_result.append(aug_result)
                used_augmenters.append(name)
            print(f"Concluded {name} ({aug_counter}/{len(augmenters)}; {round(time.time()-aug_start_time,2)} seconds).")
        except:
            print(f"Failed {name} ({aug_counter}/{len(augmenters)}; {round(time.time()-aug_start_time,2)} seconds).")
    print(f"Row augmented ({round(time.time()-row_start_time,2)} seconds).")
    aug_counter = 0
    augmented_row = {}
    augmented_row.update(row)
    augmented_row.update({"augmentation":final_result})
    augmented_row.update({"augmenters": used_augmenters})
    aug_data.update(augmented_row)
    if(row_counter == 1):
        csv_file = open('/path/to/annoMI_therapist_augmented_WITH_AUGMENT_INFO.csv', mode='w')
        fieldnames = [el for el in augmented_row.keys()]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(augmented_row)
        csv_file.close()   
    else:
        csv_file = open('/path/to/annoMI_therapist_augmented_WITH_AUGMENT_INFO.csv', mode='a')
        fieldnames = [el for el in augmented_row.keys()]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writerow(augmented_row)
        csv_file.close()
        



In [ ]:
'''Saving Anno-AUGMI'''
aug = pd.read_csv('path/to/annoMI_therapist_augmented_WITH_AUGMENT_INFO.csv')

final = []
for index, row in aug.iterrows():
    aug_list = ast.literal_eval(row['augmentation'])
    for single_augmentation in aug_list:
        new_row = copy.deepcopy(row)
        new_row['utterance_text'] = str(single_augmentation)
        final.append(new_row)

df_aug = pd.DataFrame(final)
anno_augmi = df.append(df_aug, ignore_index=True)
anno_augmi = anno_augmi[['utterance_text','mi_quality','topic']]
anno_augmi.to_csv('path/to/anno_augmi_therapist_only.csv')

<h3>Pre-processing (necessary for Fairness-aware augmentation only)</h3>
The next augmentation takes a sensitive variable as the target. In this case we chose therapy topics, meaning that we will try to obtain the same amount of HQ and LQ utterances regardless of the topics. If this is impossible for some topics (e.g.: there are no LQ or no HQ utterances at all) they will be automatically excluded.

This procedure results in the <b>Anno-FairMI</b> dataset

In [ ]:
df= pd.read_csv("path/to/anno_mi_therapist_only.csv")

In [11]:
#aggregating mi_quality sentences by topic (to inspect topics that should be cut out)
out = (df.groupby('topic')['mi_quality']
   .value_counts()
   .sort_index()
   .to_frame(name='Utterances no.')
).reset_index()

In [12]:
out

,topic,mi_quality,Utterances no.
0,Being assertive with flatmate about moving out,1,18
1,anxiety management,1,27
2,asthma management,0,9
3,asthma management,1,52
4,avoiding DOI,1,89
5,better oral health,0,10
6,better oral health,1,13
7,birth control,1,10
8,changing approach to disease,1,60
9,charging battery,1,12


In [13]:
#moving in dict to exclude topics that lacks high or low quality examples
topics_stats = {}
for row in out.iterrows():
    topic = row[1][0]
    quality = row[1][1]
    count = row[1][2]
    if topic in topics_stats.keys():
        prev = topics_stats.get(topic)
        hi = prev['hi']
        lo = prev['lo']
        topics_stats.update({topic: {"hi":hi+count, "lo":lo}} if quality == 1 else {topic: {"hi":hi, "lo":lo+count}})
    else:
        topics_stats.update({topic: {"hi":count, "lo":0}} if quality == 1 else {topic: {"hi":0, "lo":count}})

In [14]:
usable_topics = dict(filter(lambda x: x[1]['hi'] != 0 and x[1]['lo'] != 0, 
                      topics_stats.items()))
pp.pprint(usable_topics)

{'asthma management': {'hi': 52, 'lo': 9},
 'better oral health': {'hi': 13, 'lo': 10},
 'compliance with rules': {'hi': 50, 'lo': 5},
 'diabetes management': {'hi': 133, 'lo': 4},
 'more exercise / increasing activity': {'hi': 113, 'lo': 3},
 'providing information on medicines': {'hi': 4, 'lo': 3},
 'reducing alcohol consumption': {'hi': 413, 'lo': 99},
 'smoking cessation': {'hi': 89, 'lo': 47},
 'smoking cessation ': {'hi': 142, 'lo': 4},
 'smoking cessation; reducing alcohol consumption': {'hi': 15, 'lo': 4},
 'taking medicine / following medical procedure': {'hi': 106, 'lo': 31}}


In [15]:
useless_topics = dict(filter(lambda x: x[1]['hi'] == 0 or x[1]['lo'] == 0, 
                      topics_stats.items()))
pp.pprint(useless_topics)

{'Being assertive with flatmate about moving out': {'hi': 18, 'lo': 0},
 'anxiety management': {'hi': 27, 'lo': 0},
 'avoiding DOI': {'hi': 89, 'lo': 0},
 'birth control': {'hi': 10, 'lo': 0},
 'changing approach to disease': {'hi': 60, 'lo': 0},
 'charging battery': {'hi': 12, 'lo': 0},
 'completion of community service': {'hi': 16, 'lo': 0},
 'diagnosis': {'hi': 0, 'lo': 5},
 'diet; reducing alcohol consumption; diabetes management': {'hi': 18, 'lo': 0},
 'engaging in community activities': {'hi': 4, 'lo': 0},
 'increasing activity; taking medicine / following medical procedure': {'hi': 31,
                                                                        'lo': 0},
 'increasing self-confidence': {'hi': 18, 'lo': 0},
 'managing life': {'hi': 24, 'lo': 0},
 'more exercise / increasing activity; weight loss': {'hi': 29, 'lo': 0},
 'not getting into a car with someone who is under the influence of drugs or alcohol': {'hi': 0,
                                                        

In [16]:
#filtering topics
df = df[df['topic'].isin(usable_topics.keys())]

In [17]:
df

,Unnamed: 0,mi_quality,topic,utterance_text
0,0,1,reducing alcohol consumption,Thanks for filling it out. We give this form t...
1,1,1,reducing alcohol consumption,"So, let's see. It looks that you put-- You dri..."
2,2,1,reducing alcohol consumption,Okay. That's at least 12 drinks a week.
3,3,1,reducing alcohol consumption,"Uh, what else can you tell me about your drink..."
4,4,1,reducing alcohol consumption,"Okay. So, can I share with you some informatio..."
...,...,...,...,...
2489,2489,0,smoking cessation,Okay. So a patch is something you've already s...
2490,2490,0,smoking cessation,"-and, again, I'll get some out and I'll show y..."
2491,2491,0,smoking cessation,And that's exactly one of the symptoms of nico...
2492,2492,0,smoking cessation,"So, I would strongly recommend that you have t..."


In [18]:
'''must be assessed by looking at the output above: in our case we chose therapy topic and
the highest available number of utterances (either HQ or LQ, the highest) is 413 for <<reducing alcohol consumption>>'''
target = 413

<h3> Fairness aware augmentation (augmenting to balance target variable for each sensitive variable group) </h3>

In [19]:
augmentation_result = []
aug_counter = 0
row_counter = 0
aug_data = {}
print("Beginning augmentation")

hi_augmentations = []
lo_augmentations = []
for category in list(usable_topics.items()):
    topic = category[0]
    hi = category[1].get('hi')
    lo = category[1].get('lo')
    hi_completed = False
    lo_completed = False
    hi_augmentations = []
    lo_augmentations = []
    while(not (hi_completed and lo_completed)):
        row_counter = 1
        rows = df[df['topic'] == topic]
        
        for row in rows.iterrows():
            row_counter += 1
            utterance = row[1]['utterance_text']
            quality = row[1]['mi_quality']

            if hi_completed and lo_completed:
                break
            elif (quality==1 and hi_completed) or (quality==0 and lo_completed):
                print(f'skipping row {row_counter}: quality target reached')
                continue

            print(f'Working on topic "{topic}"')
            print(f"Augmenting row {row_counter}/{len(rows)}")
            print(f"\n{utterance}\n")
            row_start_time = time.time()
            aug_counter = 0
            for augmenter, name in augmenters:
                aug_counter += 1
                aug_start_time = time.time()
                try:
                    aug_result = augmenter.generate(utterance)
                    if type(aug_result) == list:
                        for aug_utterance in aug_result:
                            if aug_utterance != utterance:
                                aug_utterance = aug_utterance.replace('\\','')
                                aug_utterance = re.sub('\+', ' ', aug_utterance)
                                augmented_row = copy.deepcopy(row[1])
                                augmented_row['utterance_text'] = aug_utterance
                                if quality == 1:
                                    hi_augmentations.append(augmented_row)
                                else:
                                    lo_augmentations.append(augmented_row)
                    elif aug_result != utterance:
                        aug_result = aug_result.replace('\\','')
                        augmented_row = copy.deepcopy(row[1])
                        augmented_row['utterance_text'] = aug_utterance
                        if quality == 1:
                            hi_augmentations.append(augmented_row)
                        else:
                            lo_augmentations.append(augmented_row)
                    print(f"Concluded {name} ({aug_counter}/{len(augmenters)}; {round(time.time()-aug_start_time,2)} seconds).")
                except:
                    print(f"Failed {name} ({aug_counter}/{len(augmenters)}; {round(time.time()-aug_start_time,2)} seconds).")
            print(f"Row augmented ({round(time.time()-row_start_time,2)} seconds).")
            
            print(f'For current topic: ')
            print(f'\t - started from {len(rows)}')
            print(f'\t - HQ examples at this point: {len(hi_augmentations) + hi}')
            print(f'\t - LQ examples at this point: {len(lo_augmentations) + lo}')
            print(f'\t - Overall, {len(lo_augmentations) + lo + len(hi_augmentations) + hi}/{target*2}')
            if len(hi_augmentations) + hi >= target:
                print(f'-> HIGH QUALITY TARGET REACHED')
            if len(lo_augmentations) + lo >= target:
                print(f'-> LOW QUALITY TARGET REACHED')
            
            if len(hi_augmentations) + hi >= target:
                hi_completed = True
            if len(lo_augmentations) + lo >= target:
                lo_completed = True
                
        if not hi_completed:
            while (len(hi_augmentations) + hi) < target:
                hi_augmentations += copy.deepcopy(hi_augmentations)
            if (len(hi_augmentations)+hi) > target:
                while (len(hi_augmentations) + hi) > target:
                    hi_augmentations.pop()
            hi_completed = True
        if not lo_completed:
            while (len(lo_augmentations) + lo) < target:
                lo_augmentations += copy.deepcopy(lo_augmentations)
            if (len(lo_augmentations)+lo) > target:
                while (len(lo_augmentations) + lo) > target:
                    lo_augmentations.pop()
            lo_completed = True
        if hi_completed and lo_completed:
            df = df.append(hi_augmentations, ignore_index=True)
            df = df.append(lo_augmentations, ignore_index=True)
            
            
    

Beginning augmentation
Working on topic "asthma management"
Augmenting row 2/61

Hi, Sal. Thanks for coming in today.

Failed FactiveVerb (1/10; 0.01 seconds).


/home/uccollab/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2318: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


Concluded Formal2Casual (2/10; 1.18 seconds).
Failed Hypernym substitution (3/10; 0.84 seconds).
Concluded Basic style (4/10; 11.69 seconds).
Concluded Tweet style (5/10; 11.4 seconds).
Failed Synonym substitution (6/10; 0.01 seconds).


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Concluded Backtranslation (7/10; 1.79 seconds).
Concluded Filler Word (8/10; 0.0 seconds).


...

Concluded ProtAugment (9/10; 1.61 seconds).
Concluded Slangificator (10/10; 0.01 seconds).
Row augmented (26.23 seconds).
For current topic: 
	 - started from 137
	 - HQ examples at this point: 312
	 - LQ examples at this point: 31
	 - Overall, 343/826
Working on topic "taking medicine / following medical procedure"
Augmenting row 20/137

Okay, um, so if so where we are right now if I understand is you're feeling more comfortable about the concern about muscle weakness?

Failed FactiveVerb (1/10; 0.01 seconds).
Concluded Formal2Casual (2/10; 2.42 seconds).
Failed Hypernym substitution (3/10; 0.57 seconds).
Concluded Basic style (4/10; 8.27 seconds).
Concluded Tweet style (5/10; 8.22 seconds).
Failed Synonym substitution (6/10; 0.01 seconds).
Concluded Backtranslation (7/10; 3.28 seconds).
Concluded Filler Word (8/10; 0.0 seconds).
Concluded ProtAugment (9/10; 1.05 seconds).
Concluded Slangificator (10/10; 0.01 seconds).
Row augmented (23.82 seconds).
For current topic: 
	 - started fro

Concluded Backtranslation (7/10; 8.35 seconds).
Concluded Filler Word (8/10; 0.0 seconds).
Concluded ProtAugment (9/10; 4.04 seconds).
Concluded Slangificator (10/10; 0.01 seconds).
Row augmented (39.24 seconds).
For current topic: 
	 - started from 137
	 - HQ examples at this point: 422
	 - LQ examples at this point: 31
	 - Overall, 453/826
-> HIGH QUALITY TARGET REACHED
skipping row 30: quality target reached
Working on topic "taking medicine / following medical procedure"
Augmenting row 31/137

Uh, I'm here to talk to you about, uh, your cholesterol and the, um, medicine your doctor prescribed.

Concluded FactiveVerb (1/10; 0.01 seconds).
Concluded Formal2Casual (2/10; 2.4 seconds).
Failed Hypernym substitution (3/10; 0.57 seconds).
Concluded Basic style (4/10; 8.28 seconds).
Concluded Tweet style (5/10; 8.21 seconds).
Failed Synonym substitution (6/10; 0.01 seconds).
Concluded Backtranslation (7/10; 3.47 seconds).
Concluded Filler Word (8/10; 0.0 seconds).
Concluded ProtAugment (9/

Concluded Formal2Casual (2/10; 1.68 seconds).
Failed Hypernym substitution (3/10; 0.57 seconds).
Concluded Basic style (4/10; 11.59 seconds).
Concluded Tweet style (5/10; 11.56 seconds).
Failed Synonym substitution (6/10; 0.01 seconds).
Concluded Backtranslation (7/10; 2.2 seconds).
Concluded Filler Word (8/10; 0.0 seconds).
Concluded ProtAugment (9/10; 0.88 seconds).
Concluded Slangificator (10/10; 0.01 seconds).
Row augmented (28.5 seconds).
For current topic: 
	 - started from 137
	 - HQ examples at this point: 422
	 - LQ examples at this point: 99
	 - Overall, 521/826
-> HIGH QUALITY TARGET REACHED
Working on topic "taking medicine / following medical procedure"
Augmenting row 114/137

Okay. It looks like cholesterol statin. Do you know how you're supposed to take the medication?

Concluded FactiveVerb (1/10; 0.01 seconds).
Concluded Formal2Casual (2/10; 1.82 seconds).
Failed Hypernym substitution (3/10; 0.58 seconds).
Concluded Basic style (4/10; 12.54 seconds).
Concluded Tweet st

Concluded Formal2Casual (2/10; 3.1 seconds).
Failed Hypernym substitution (3/10; 0.58 seconds).
Concluded Basic style (4/10; 12.85 seconds).
Concluded Tweet style (5/10; 12.84 seconds).
Failed Synonym substitution (6/10; 0.01 seconds).
Concluded Backtranslation (7/10; 8.52 seconds).
Concluded Filler Word (8/10; 0.0 seconds).
Concluded ProtAugment (9/10; 3.82 seconds).
Concluded Slangificator (10/10; 0.01 seconds).
Row augmented (41.76 seconds).
For current topic: 
	 - started from 137
	 - HQ examples at this point: 422
	 - LQ examples at this point: 210
	 - Overall, 632/826
-> HIGH QUALITY TARGET REACHED
Working on topic "taking medicine / following medical procedure"
Augmenting row 124/137

Okay. What did the doctor tell you cholesterol statin is for?

Concluded FactiveVerb (1/10; 0.01 seconds).
Concluded Formal2Casual (2/10; 1.44 seconds).
Failed Hypernym substitution (3/10; 0.58 seconds).
Concluded Basic style (4/10; 11.58 seconds).
Concluded Tweet style (5/10; 11.57 seconds).
Faile

Concluded Formal2Casual (2/10; 1.46 seconds).
Failed Hypernym substitution (3/10; 0.59 seconds).
Concluded Basic style (4/10; 11.56 seconds).
Concluded Tweet style (5/10; 11.58 seconds).
Failed Synonym substitution (6/10; 0.01 seconds).
Concluded Backtranslation (7/10; 1.66 seconds).
Concluded Filler Word (8/10; 0.0 seconds).
Concluded ProtAugment (9/10; 0.71 seconds).
Concluded Slangificator (10/10; 0.01 seconds).
Row augmented (27.58 seconds).
For current topic: 
	 - started from 137
	 - HQ examples at this point: 422
	 - LQ examples at this point: 316
	 - Overall, 738/826
-> HIGH QUALITY TARGET REACHED
Working on topic "taking medicine / following medical procedure"
Augmenting row 133/137

All right, and he's right. This medication will help lower your cholesterol, and it will help lower your risk of having a heart attack or a stroke. How did the doctor tell you to take this medicine?

Concluded FactiveVerb (1/10; 0.01 seconds).
Concluded Formal2Casual (2/10; 2.67 seconds).
Failed H

In [21]:
#aggregating mi_quality sentences by topic (to inspect topics that should be cut out)
out = (df.groupby('topic')['mi_quality']
   .value_counts()
   .sort_index()
   .to_frame(name='Utterances no.')
).reset_index()
out

,topic,mi_quality,Utterances no.
0,asthma management,0,413
1,asthma management,1,415
2,better oral health,0,413
3,better oral health,1,413
4,compliance with rules,0,413
5,compliance with rules,1,423
6,diabetes management,0,413
7,diabetes management,1,422
8,more exercise / increasing activity,0,413
9,more exercise / increasing activity,1,415


In [22]:
df.to_csv('path/to/anno_fairmi_therapist_only.csv')